In [1]:
import pandas as pd
from os.path import join

In [2]:
data_path = "/media/aviral/8018AF4F18AF4350/Documents and Settings/joavi/Documents/B-TECH/7th SEM/AML/Hackathon/Dataset-1/selfie_dataset.txt"#join("..", "..", "Dataset-1", "selfie_dataset.txt")
image_path = "/media/aviral/8018AF4F18AF4350/Documents and Settings/joavi/Documents/B-TECH/7th SEM/AML/Hackathon/Dataset-1/images"#join("..", "..", "Dataset-1", "selfie_dataset.txt")#join("..", "..", "Dataset-1", "images")

In [3]:
headers = [
    "image_name", "score", "partial_faces" ,"is_female" ,"baby" ,"child" ,"teenager" ,"youth" ,"middle_age" ,"senior" ,"white" ,"black" ,"asian" ,"oval_face" ,"round_face" ,"heart_face" ,"smiling" ,"mouth_open" ,"frowning" ,"wearing_glasses" ,"wearing_sunglasses" ,"wearing_lipstick" ,"tongue_out" ,"duck_face" ,"black_hair" ,"blond_hair" ,"brown_hair" ,"red_hair" ,"curly_hair" ,"straight_hair" ,"braid_hair" ,"showing_cellphone" ,"using_earphone" ,"using_mirror", "braces" ,"wearing_hat" ,"harsh_lighting", "dim_lighting"
]
df_image_details = pd.read_csv(data_path, names=headers, delimiter=" ")
df_image_details.head(3)

,image_name,score,partial_faces,is_female,baby,child,teenager,youth,middle_age,senior,...,curly_hair,straight_hair,braid_hair,showing_cellphone,using_earphone,using_mirror,braces,wearing_hat,harsh_lighting,dim_lighting
0,00a454da495e11e28a7322000a1fa414_6,3.901,1,1,-1,-1,-1,1,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
1,00cddb96ac4c11e3a30212279ba1b65f_6,4.385,1,1,-1,-1,-1,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
2,01cdd7aa1a1a11e2aaa822000a1fb0dd_6,4.243,-1,1,-1,-1,1,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1


In [4]:
df_image_details = df_image_details[df_image_details.is_female != 0]
df_image_details.replace(to_replace=-1, value=0, inplace=True)

In [5]:
image_names = df_image_details.image_name.values.copy()
image_scores = df_image_details[headers[1]].values.copy()
image_attrs = df_image_details[headers[2:]].values.copy()

In [6]:
image_paths = [join(image_path, iname) + '.jpg' for iname in image_names]

In [7]:
from keras.utils import Sequence
import numpy as np
import cv2

/mnt/Xtension/Ubuntu_Files/conda_virtual_envs/py3.6/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [36]:
class ImageGenerator(Sequence):
    def __init__(self, x_set, y_set, batch_size):
        self.x, self.y = x_set, y_set
        self.batch_size = batch_size

    def __len__(self):
        return int(np.ceil(len(self.x) / float(self.batch_size)))

    def __getitem__(self, idx):
        batch_x = self.x[idx * self.batch_size:(idx + 1) * self.batch_size]
        batch_y1 = self.y[0][idx * self.batch_size:(idx + 1) * self.batch_size]
        batch_y2 = self.y[1][idx * self.batch_size:(idx + 1) * self.batch_size]

        # read your data here using the batch lists, batch_x and batch_y
        x = [self.read_image(filename) for filename in batch_x] 
        y1 = [atrributes for atrributes in batch_y1]
        y2 = [atrributes for atrributes in batch_y2]
        return [np.array(x), np.array(x)], [np.array(y1), np.array(y2)]
    
    def read_image(self, fname):
        im = cv2.imread(fname)
        im = cv2.resize(im, (224, 224))
        return im;

# Training Model


In [16]:
from keras.applications import resnet50
from keras.layers import Dense, Conv2D, MaxPool2D, Input, Flatten, concatenate
from keras.models import Model, load_model

In [10]:
model_rnet = load_model("resnet50.hdf5")

Instructions for updating:
keep_dims is deprecated, use keepdims instead


/mnt/Xtension/Ubuntu_Files/conda_virtual_envs/py3.6/lib/python3.6/site-packages/keras/models.py:251: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '


In [11]:
model_rnet.layers.pop()

In [12]:
model_classification = Dense(36, activation='softmax', name='classification')(model_rnet.output)

In [29]:
model_regression_input = Input((224, 224, 3))
model_regression = Conv2D(16, 5, strides=(2, 2))(model_regression_input)
model_regression = MaxPool2D()(model_regression)

model_regression = Conv2D(24, 5, strides=(2, 2))(model_regression)
model_regression = MaxPool2D()(model_regression)

model_regression = Conv2D(32, 5, strides=(2, 2))(model_regression)
model_regression = MaxPool2D()(model_regression)

model_regression = Flatten()(model_regression)
model_regression = Dense(128)(model_regression)

model_regression = concatenate([model_regression, model_classification])

model_regression = Dense(1, name='regression')(model_regression)

In [30]:
model = Model(inputs=[model_rnet.input, model_regression_input], outputs=[model_classification, model_regression])

In [31]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 112, 112, 64) 256         conv1[0][0]                      
__________________________________________________________________________________________________
activation

In [32]:
model.compile(
    optimizer='adam',
    loss={
        'regression': 'mean_squared_error',
        'classification': 'binary_crossentropy'
    },
    metrics=[
        'accuracy'
    ]
)

In [37]:
train_gen = ImageGenerator(image_paths, (image_attrs, image_scores), batch_size=8)
test_gen = ImageGenerator(image_paths, (image_attrs, image_scores), batch_size=8)

In [38]:
model.fit_generator(train_gen, epochs=200, steps_per_epoch=16)

Epoch 1/200
16/16 [==============================] - 30s 2s/step - loss: 6231.6016 - classification_loss: 0.6511 - regression_loss: 6230.9505 - classification_acc: 0.8249 - regression_acc: 0.0000e+00
Epoch 2/200
16/16 [==============================] - 13s 829ms/step - loss: 339.2476 - classification_loss: 0.5511 - regression_loss: 338.6965 - classification_acc: 0.8529 - regression_acc: 0.0000e+00
Epoch 3/200
16/16 [==============================] - 13s 828ms/step - loss: 110.2443 - classification_loss: 0.5325 - regression_loss: 109.7118 - classification_acc: 0.8581 - regression_acc: 0.0000e+00
Epoch 4/200
16/16 [==============================] - 13s 843ms/step - loss: 82.9318 - classification_loss: 0.6284 - regression_loss: 82.3034 - classification_acc: 0.8309 - regression_acc: 0.0000e+00
Epoch 5/200
16/16 [==============================] - 13s 833ms/step - loss: 59.5847 - classification_loss: 0.6047 - regression_loss: 58.9800 - classification_acc: 0.8375 - regression_acc: 0.0000e+00


KeyboardInterrupt: 